# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
import collections
from collections import Counter
from operator import itemgetter

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
text = open('/content/drive/My Drive/Colab Notebooks/peace.txt', 'r', encoding = 'utf-8').read()[2:]
len(text)

3227579

In [42]:
import string
import re

def preprocess_text(text):
    # TODO
    # make lowercase
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    
    text = text.lower()
    punct = r'[\(\)\*\,\-\/\:\;\=!\?#$%@\[\]\—\‘\’\“\”]'
    mult_dot = r'\.{2,}'
    respace = r' {2,}'
    respace_1 = r'\n '
    respace_2 = r' \.'
    re_newstr = r'\n{2,}'
    text = re.sub(punct, ' ', text)
    text = re.sub(mult_dot, '.', text)
    text = re.sub(respace, ' ', text)
    text = re.sub(respace_1, '\n', text)
    text = re.sub(respace_2, '.', text)
    text = re.sub(re_newstr, '\n', text)
    return text

text = preprocess_text(text)
#assert len(text) == 3141169
len(text)

3143402

In [0]:
text = re.sub('\n', '', text)
text = text.split('.')
text = [x.strip() for x in text]

In [45]:
text[:10]

['the project gutenberg ebook of war and peace by leo tolstoythis ebook is for the use of anyone anywhere at no cost and with almostno restrictions whatsoever',
 'you may copy it give it away or re useit under the terms of the project gutenberg license included with thisebook or online at www',
 'gutenberg',
 'orgtitle war and peaceauthor leo tolstoytranslators louise and aylmer maudeposting date january 10 2009 ebook 2600 last updated january 21 2019language englishcharacter set encoding utf 8start of this project gutenberg ebook war and peace an anonymous volunteer and david widgerwar and peaceby leo tolstoy tolstoicontentsbook one 1805chapter ichapter iichapter iiichapter ivchapter vchapter vichapter viichapter viiichapter ixchapter xchapter xichapter xiichapter xiiichapter xivchapter xvchapter xvichapter xviichapter xviiichapter xixchapter xxchapter xxichapter xxiichapter xxiiichapter xxivchapter xxvchapter xxvichapter xxviichapter xxviiibook two 1805chapter ichapter iichapter iiic

In [0]:
#from collections import Counter
import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        items = list([i for sent in text for i in sent])
        self.itos = list(set(items))
        self.stoi = {letter: idx for idx, letter in enumerate(self.itos)}
        new_text = []
        for sent in text:
          new_text.append([self.stoi[letter] for letter in sent])
        text = new_text
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigrams = collections.Counter()
            for sent in text:
              for i in range(len(sent)):
                if i + 1 < len(sent):
                  bigrams[(sent[i], sent[i+1])] += 1
            
            mostcom_bi = bigrams.most_common(1)[0][0]
            f_of_bi = str(self.itos[int(mostcom_bi[0])])
            s_of_bi = str(self.itos[int(mostcom_bi[1])])
            new_token = f_of_bi + s_of_bi
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            new_text = []
            for sent in text:
              new_sent = []
              jump = -1
              for i in range(len(sent)):
                if i + 1 < len(sent):
                  if i != jump:
                    if new_token == self.itos[sent[i]] + self.itos[sent[i+1]]:
                      new_sent.append(new_id)
                      jump = i + 1
                    else:
                      new_sent.append(sent[i])
                else: 
                  if i != jump:
                    new_sent.append(sent[i])                    
              new_text.append(new_sent)
              
            text = new_text
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        # TODO tokenize text by symbols using self.stoi
        new_text0 = []
        for sent in text:
          new_text0.append([self.stoi[letter] for letter in sent])
        text = new_text0
        
        # find occurences of the token in the text and replace them with token_id
        for token_id, token in enumerate(self.itos):
          new_text = []
          for sent in text:
            new_sent = []
            jump = -1
            for i in range(len(sent)):
              if i + 1 < len(sent):
                if i != jump:
                  if token == self.itos[sent[i]] + self.itos[sent[i+1]]:
                    new_sent.append(token_id)
                    jump = i + 1
                  else:
                    new_sent.append(sent[i])
              else: 
                if i != jump:
                  new_sent.append(sent[i])                    
            new_text.append(new_sent)
              
            text = new_text
       
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        if isinstance(tok, int):
          result = self.itos[tok]
        else:
          result = []
          for i in token:
            result.append(self.itos[i])
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size 
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = Counter() # TODO create array for storing 3-gram counters
        self.proba_ab = Counter()
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = []
        for item in range(self.vocab_size):
            result.append(self.get_proba(a, b, item, tau))
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        c_abc = self.proba[a, b, c]     
        c_ab = self.proba_ab[a, b]
        result = ((c_abc + self.delta) ** (1/tau))/((c_ab + self.delta) ** (1/tau))
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        for sent in text:
            for i in range(len(sent)):
              if i <= len(sent) - 3:
                self.proba[sent[i], sent[i+1], sent[i+2]] += 1                     
        # TODO count 3-grams in the text
        
        for sent in text:
            for i in range(len(sent)):
              if i <= len(sent) - 2:
                self.proba_ab[sent[i], sent[i+1]] += 1 
        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    
    beam = [(input_seq, 1)]
    for i in range(max_len):
      candidates = []
      candidates_proba = []
      
      for tup in beam:
        seq = tup[0]
        if seq[-2] != end_token:
          best_k = []
          prob = lm.infer(seq[-2], seq[-1], tau)
          sort_prob = sorted([(p, idx) for idx, p in enumerate(prob)], reverse = True)
          for idx in sort_prob[:k]:
            if idx[1] < end_token:
              best_k.append(bpe.itos[idx[1]]) 
          for tok in best_k:
            candidates.append(seq + [bpe.stoi[tok]])
            candidates_proba.append(sort_prob[bpe.stoi[tok]][0])      
      beam = [(candidates[u], candidates_proba[u]) for u in range(k)]
  
    return beam
    

In [121]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for res in result:
  print(bpe.decode(tuple([i for i in res[0]])), res[1])

horse was sold not b 5.102756064303754e-15
horse was sold not s 6.599707150644614e-20
horse was sold not to  9.251801682952108e-38
horse was sold not l 1.2754394268978178e-26
horse was sold not k 1.0826425087923193e-15
horse was sold not re 9.251801682952108e-38
horse was sold not u 4.842240858492817e-17
horse was sold not f 1.0453862326160329e-11
horse was sold not m 9.473844923342959e-35
horse was sold not c 1.302161333895674e-15


In [123]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for res in result:
  print(bpe.decode(tuple([i for i in res[0]])), res[1])

here was sold not  7.007302040006144e-39
here was sold not 2.5019342280897436e-26
here was sold noth 7.007302040006144e-39
here was sold nov 7.007302040006144e-39
here was sold nob 1.4812895690103165e-20
here was sold noi 7.007302040006144e-39
here was sold nos 7.175477288966291e-36
here was sold noc 8.341662193345087e-22
here was sold nod 4.137741781603228e-34
here was sold nol 7.007302040006144e-39


In [124]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities
for res in result:
  print(bpe.decode(tuple([i for i in res[0]])), res[1])

whatess mary to the cou 0.0003184713375796178
whatess mary to the con 0.0003184713375796178
whatess mary to the ca 0.0035031847133757963
whatess mary to the com 0.0003184713375796178
whatess mary to the co 0.0070063694267515925
whatess mary to the cr 0.0003184713375796178
whatess mary to the chi 0.0003184713375796178
whatess mary to the cl 0.0003184713375796178
whatess mary to the car 0.0003184713375796178
whatess mary to the cor 0.0003184713375796178


In [125]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for res in result:
  print(bpe.decode(tuple([i for i in res[0]])), res[1])

gun and so musside  1.5074038602896436e-36
gun and so mussiden 1.5074038602896436e-36
gun and so musside 8.901069054624316e-32
gun and so mussider 1.5074038602896436e-36
gun and so mussidl 8.901069054624316e-32
gun and so mussided  1.5074038602896436e-36
gun and so mussidg 8.692450248656558e-25
gun and so mussidd 4.360235835175713e-25
gun and so mussidn 1.6971858658743216e-21
gun and so mussiding  1.5074038602896436e-36


In [127]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    p = 1
    for i in range(len(snt)):
      if i <= len(snt) - 3:
        p *= lm.get_proba(snt[i], snt[i + 1], snt[i + 2])
    m = -1 / len(snt)
    result = p ** m #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)

12.472877566841332